- https://www.gso.gov.vn/px-web-2/?pxid=V1016&theme=Gi%C3%A1o%20d%E1%BB%A5c
- https://www.gso.gov.vn/px-web-2/?pxid=V1004&theme=Gi%C3%A1o%20d%E1%BB%A5c
- https://data.moet.gov.vn/index.php/s/LEeizeiFLDdX67t#pdfviewer
- https://data.moet.gov.vn/index.php/s/TEABYvhDO4c7zfg#pdfviewer
- https://data.moet.gov.vn/index.php/s/D76oJngZbjavhYH#pdfviewer
---
- https://api.worldbank.org/v2/country/vn/indicator/NY.GDP.MKTP.KD.ZG?format=json&per_page=1000&date=1985:2022
- https://api.worldbank.org/v2/country/vn/indicator/NY.GDP.MKTP.CD?format=json&per_page=1000&date=1985:2022
- https://api.worldbank.org/v2/country/vn/indicator/NY.GDP.PCAP.PP.CD?locations=VN
---
- https://api.worldbank.org/v2/country/vn/indicator/SP.POP.TOTL?format=json&per_page=1000
- https://api.worldbank.org/v2/country/vn/indicator/SP.RUR.TOTL?format=json&per_page=1000
- https://api.worldbank.org/v2/country/vn/indicator/SP.URB.TOTL?format=json&per_page=1000
- https://api.worldbank.org/v2/country/vn/indicator/SP.POP.GROW?format=json&per_page=1000
- https://api.worldbank.org/v2/country/vn/indicator/SP.DYN.CBRT.IN?format=json&per_page=1000

#### GDPcompare.csv

In [53]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

code_lst = { "gdp_grow" :  "NY.GDP.MKTP.KD.ZG",
            "gdp" : "NY.GDP.MKTP.CD",
            "gdp_capital" : "NY.GDP.PCAP.PP.CD"
            } 
apis = {k:f"https://api.worldbank.org/v2/countries/vn;sg;th;id;my;us;z4;1w/indicators/{v}?format=json&per_page=1000" for k,v in code_lst.items()}

df = []

for value, api in apis.items():
    res = requests.get(api)
    js_res = res.json()[1]

    df_tmp = pd.DataFrame(js_res)
    df_tmp['country_name'] = df_tmp['country'].apply(lambda x : x['value'])
    df_tmp.drop(['country', 'indicator','countryiso3code'], axis= 1, inplace= True)

    df_tmp.insert(0, 'country_name', df_tmp.pop('country_name'))

    df_tmp = df_tmp.loc[:, 'country_name':'value']

    df_tmp.rename(columns= {'value':value, 'date': 'year'}, inplace = True)

    df.append(df_tmp)



In [55]:
pd.set_option("display.max_columns",None)

gdp_grow = pd.DataFrame(df[0]).pivot_table(index='year', columns='country_name', values='gdp_grow').reset_index()
gdp = pd.DataFrame(df[1]).pivot_table(index='year', columns='country_name', values='gdp').reset_index()
gdp_ppp = pd.DataFrame(df[2]).pivot_table(index='year', columns='country_name', values='gdp_capital').reset_index()

gdp_compare = pd.merge(gdp,pd.merge(gdp_grow, gdp_ppp, how= "outer", on='year', suffixes=(' %',' GDP PPP')), how= "outer", on='year')

gdp_compare.columns.name = None
gdp_compare['year'] = gdp_compare['year'].astype('int64')


,year,East Asia & Pacific,Indonesia,Malaysia,Singapore,Thailand,United States,Viet Nam,World,East Asia & Pacific %,Indonesia %,Malaysia %,Singapore %,Thailand %,United States %,Viet Nam %,World %,East Asia & Pacific GDP PPP,Indonesia GDP PPP,Malaysia GDP PPP,Singapore GDP PPP,Thailand GDP PPP,United States GDP PPP,Viet Nam GDP PPP,World GDP PPP
0,1960,1.545311e+11,NaN,1.916229e+09,7.047517e+08,2.760751e+09,5.433000e+11,NaN,1.381135e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1961,1.551273e+11,NaN,1.901856e+09,7.646298e+08,3.034038e+09,5.633000e+11,NaN,1.446356e+12,3.768423,5.740646,7.597994,8.137530,5.362146,2.300000,NaN,3.772924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1962,1.583437e+11,NaN,2.001490e+09,8.262392e+08,3.308913e+09,6.051000e+11,NaN,1.546369e+12,5.498926,1.841978,6.421030,7.553444,7.554254,6.100000,NaN,5.375360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1963,1.765917e+11,NaN,2.510110e+09,9.176080e+08,3.540403e+09,6.386000e+11,NaN,1.670666e+12,7.816489,-2.237030,7.338804,10.040173,7.999831,4.400000,NaN,5.194927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1964,2.025624e+11,NaN,2.674424e+09,8.941533e+08,3.889130e+09,6.858000e+11,NaN,1.832616e+12,10.536636,3.529698,5.358963,-3.103168,6.830996,5.800000,NaN,6.564476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,2018,2.648785e+13,1.042272e+12,3.587888e+11,3.768695e+11,5.067542e+11,2.053306e+13,3.101065e+11,8.654268e+13,4.760219,5.174292,4.843087,3.575433,4.222870,2.945385,7.465007,3.276283,17439.983434,11671.432330,27480.954923,100642.752176,18093.819783,62823.309438,9867.668608,16991.375639
59,2019,2.703267e+13,1.119100e+12,3.651777e+11,3.768376e+11,5.439767e+11,2.138098e+13,3.343653e+11,8.777740e+13,3.960591,5.019288,4.413187,1.331261,2.114558,2.294439,7.359263,2.590785,18418.865244,12360.691736,28848.100854,102630.933142,18760.412403,65120.394663,10686.803020,17720.378005
60,2020,2.715525e+13,1.059055e+12,3.374562e+11,3.483921e+11,5.004573e+11,2.106047e+13,3.466157e+11,8.527268e+13,-0.147498,-2.065512,-5.456847,-3.901053,-6.066926,-2.767803,2.865413,-3.057810,18631.301790,12160.716762,27300.266796,100226.368465,17810.250753,63528.634303,11035.999595,17363.881585
61,2021,3.114127e+13,1.186505e+12,3.738324e+11,4.237971e+11,5.055681e+11,2.331508e+13,3.661376e+11,9.715318e+13,6.211591,3.703055,3.297641,8.882354,1.492095,5.945485,2.561564,6.228594,20531.190225,13086.392418,29139.370495,118887.822066,18855.023273,70219.472454,11727.796224,19068.841902


### pop.csv

In [80]:
import requests
import pandas as pd

apis = ['https://api.worldbank.org/v2/country/vn/indicator/SP.POP.TOTL?format=json&per_page=1000',
        'https://api.worldbank.org/v2/country/vn/indicator/SP.RUR.TOTL?format=json&per_page=1000',
        'https://api.worldbank.org/v2/country/vn/indicator/SP.URB.TOTL?format=json&per_page=1000',
        'https://api.worldbank.org/v2/country/vn/indicator/SP.POP.GROW?format=json&per_page=1000',
        'https://api.worldbank.org/v2/country/vn/indicator/SP.DYN.CBRT.IN?format=json&per_page=1000']

df_list = []
nameList = []
for api in apis:
        pop_res = requests.get(api)
        pop_js = pop_res.json()[1]
        df = pd.DataFrame(pop_js)
        name = df['indicator'].apply(lambda x:x['value'])[0]
        df.rename(columns= {'date': 'year', 'value':  name}, inplace= True)
        df = df[['year', name]]

        nameList.append(df.columns[1])
        df_list.append(df)

pop_df = pd.concat(df_list, axis= 1)
# req = requests.get('https://api.worldbank.org/v2/country/vn/indicator/SP.DYN.CBRT.IN?format=json&per_page=1000')
# res = req.json()[1]
# df = pd.DataFrame(res)
# name = df['indicator'].apply(lambda x:x['value'])
nameList[-1]

'Birth rate, crude (per 1,000 people)'

In [621]:
pop = pop_df.drop('year', axis= 1)
pop.insert(0, 'year', pop_df.iloc[:, 0].astype('int64'))
pop.rename(columns= {'SP.POP.TOTL':'Total population', 'SP.RUR.TOTL': 'Rural Population', 'SP.URB.TOTL': 'Urban Population', 'SP.POP.GROW': 'Population growth %', 'SP.DYN.CBRT.IN': 'Birth rate per 1000'}, inplace = True)
pop['Newborn'] = round(pop['Total population']*(pop['Birth rate per 1000']/1000), 0)
gdp_vn = gdp_compare[['year', 'Viet Nam', 'Viet Nam %']].rename(columns={'Viet Nam': 'GDPinUSD', 'Viet Nam %': 'GDP growth %'})
pop = pop.merge(gdp_vn, on= 'year').sort_values(by= 'year').reset_index(drop= True)
pop

,year,Total population,Rural Population,Urban Population,Population growth %,Birth rate per 1000,Newborn,GDPinUSD,GDP growth %
0,1960,32718461.0,27908847.0,4809614.0,NaN,41.331,1352287.0,NaN,NaN
1,1961,33621982.0,28568262.0,5053720.0,2.724060,35.609,1197245.0,NaN,NaN
2,1962,34533889.0,29226376.0,5307513.0,2.676105,39.800,1374449.0,NaN,NaN
3,1963,35526727.0,29944057.0,5582670.0,2.834415,39.719,1411086.0,NaN,NaN
4,1964,36509166.0,30644334.0,5864832.0,2.727807,38.366,1400711.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
58,2018,94914330.0,60822052.0,34092278.0,0.932840,15.873,1506575.0,3.101065e+11,7.465007
59,2019,95776716.0,60695620.0,35081096.0,0.904491,15.663,1500151.0,3.343653e+11,7.359263
60,2020,96648685.0,60560066.0,36088619.0,0.906299,15.400,1488390.0,3.466157e+11,2.865413
61,2021,97468029.0,60379495.0,37088534.0,0.844182,15.008,1462800.0,3.661376e+11,2.561564


### VietnameseStudent

In [117]:
import requests
def get_api(api, field):
    req  = requests.get(api)
    res = req.json()['dataset']['value']
    year = np.arange(1995, 1995 + (len(res)//3), dtype= int)
    df = pd.DataFrame({'year': year,
                       field[0]: res[0:len(year)],
                       field[1]: res[len(year):len(year)*2],
                       field[2]: res[len(year)*2:]})
    df[df.isnull()] = 0

    return df
    
edu1 = get_api('https://pxweb.gso.gov.vn:443/sq/2156adec-bb77-4bde-8c13-342161c4a474', field= ['Primary', 'Secondary', 'Highschool'])
edu2 = get_api('https://pxweb.gso.gov.vn:443/sq/66e42bba-b202-4941-9ee0-3f48fff92b81', field= ['Total_higher_edu', 'Higher_public_edu', 'Higher_private_edu'])

edu1.insert(1, 'K_12', edu1.loc[:, 'Primary':].sum(axis=1))
edu2['Total_higher_edu'] = edu2['Higher_private_edu'] + edu2['Higher_public_edu']

edu = edu2.merge(edu1, how='outer', on='year')
edu.iloc[:,1:] *= 1000

# edu.to_csv('VietnamStudent.csv')

req  = requests.get('https://pxweb.gso.gov.vn:443/sq/2156adec-bb77-4bde-8c13-342161c4a474')
dimension = req.json()['dataset']['dimension']
locate = [i for i in dimension if 'Năm' in i][0]
# semeter = dimension[locate]['category']['label']
# year = [int(value.split('-')[0].strip()) for value in semeter.values()]
# dimension['Năm học']['category']['label']
dimension[locate]['category']['label']
edu

,year,Total_higher_edu,Higher_public_edu,Higher_private_edu,K_12,Primary,Secondary,Highschool
0,1995,297900.0,297900.0,0.0,15561000.0,10228800.0,4312700.0,1019500.0
1,1996,509300.0,509300.0,0.0,16348000.0,10352700.0,4839700.0,1155600.0
2,1997,662800.0,662800.0,0.0,16970200.0,10383600.0,5204600.0,1382000.0
3,1998,682300.0,682300.0,0.0,17391100.0,10223900.0,5514300.0,1652900.0
4,1999,734900.0,734900.0,0.0,17685300.0,10033500.0,5694800.0,1957000.0
5,2000,899500.0,795600.0,103900.0,17776100.0,9741100.0,5863600.0,2171400.0
6,2001,974100.0,873000.0,101100.0,17875600.0,9315300.0,6259100.0,2301200.0
7,2002,1020700.0,908800.0,111900.0,17699600.0,8815700.0,6429700.0,2454200.0
8,2003,1131000.0,993900.0,137100.0,17505400.0,8346000.0,6569800.0,2589600.0
9,2004,1319800.0,1182000.0,137800.0,17122600.0,7744800.0,6616700.0,2761100.0


### VietnamSudent2

In [59]:
edu3 = get_api('https://pxweb.gso.gov.vn:443/sq/26d69546-6c6c-4ed6-a7af-680847767667', field= ['Total_higher_edu', 'Higher_public_edu', 'Higher_private_edu'])
edu3 = edu3.loc[(edu3['year']>2014)].set_index('year').T

edu4 = edu.loc[(edu['year'] == 2014), :'Higher_private_edu'].set_index('year').T

for y in range(2015, 2021):
    edu4[y] = edu4[y-1]*(edu3[y]/100)

edu4 = edu4.T.reset_index()
edu4['year'] = edu4['year'].astype('int32')

edu.iloc[20:, 0:4] = edu4.iloc[1:]

edu

,year,Total_higher_edu,Higher_public_edu,Higher_private_edu,K_12,Primary,Secondary,Highschool
0,1995,2.979000e+05,2.979000e+05,0.000000,15561000.0,10228800.0,4312700.0,1019500.0
1,1996,5.093000e+05,5.093000e+05,0.000000,16348000.0,10352700.0,4839700.0,1155600.0
2,1997,6.628000e+05,6.628000e+05,0.000000,16970200.0,10383600.0,5204600.0,1382000.0
3,1998,6.823000e+05,6.823000e+05,0.000000,17391100.0,10223900.0,5514300.0,1652900.0
4,1999,7.349000e+05,7.349000e+05,0.000000,17685300.0,10033500.0,5694800.0,1957000.0
5,2000,8.995000e+05,7.956000e+05,103900.000000,17776100.0,9741100.0,5863600.0,2171400.0
6,2001,9.741000e+05,8.730000e+05,101100.000000,17875600.0,9315300.0,6259100.0,2301200.0
7,2002,1.020700e+06,9.088000e+05,111900.000000,17699600.0,8815700.0,6429700.0,2454200.0
8,2003,1.131000e+06,9.939000e+05,137100.000000,17505400.0,8346000.0,6569800.0,2589600.0
9,2004,1.319800e+06,1.182000e+06,137800.000000,17122600.0,7744800.0,6616700.0,2761100.0


In [604]:
import pandas as pd

# Tạo DataFrame mẫu
data1 = {
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
}
df1 = pd.DataFrame(data1, index= [1,2,3])
print(df1)
data2 = {
    'A': [1, 2, 3],
    'B': [4, 15, 16],
    'C': [7, 18, 19],
    'D': [10, 11, 12]
}
df2 = pd.DataFrame(data2)
print(df2)
# df2[['B','C']] = df1[['B','C']] # chỉ đúng khi trùng index row
df2.iloc[1:,1:3] = df1.iloc[1:,1:]
# df1.iloc[1:,1:]

df = pd.DataFrame(df2)
df2['A'] = [11,22,33]
df1[:4]

   A  B  C
1  1  4  7
2  2  5  8
3  3  6  9
   A   B   C   D
0  1   4   7  10
1  2  15  18  11
2  3  16  19  12


,A,B,C
1,1,4,7
2,2,5,8
3,3,6,9


In [ ]:

gdp_compare.pivot(index="year",
              columns="country_name",
              values=['']).reset_index()

In [ ]:
other = pd.DataFrame({'key': ['K0', 'K1', 'K2'],
                     'B': ['B0', 'B1', 'B2'],
                     'A': ['A0', 'A1', 'A2']})

df = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3', 'K4', 'K5'],
                    'A': ['A0', 'A1', 'A2', 'A3', 'A4', 'A5']})

# df.join(other.set_index(['key','A']), on=['key','A'])

df.join(other, lsuffix='_caller', rsuffix='_other', how='inner')